In [73]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Avito 

info 
- 3 diff types of ads available to sellers on Avito: regular, highlighted, and context. 
- context ads : predict which ad will be the most relevant for (and earn the most clicks from) each potential buyer. 
- **improve on their model by predicting if individual users will click a given context ad.**
- i.e. more accurately predict click-through rates for their ads

f(x) ~ y where y is "isClick" meaning does the person has clicked on the ad  
R -> {0,1} => binary classif


## Intro

### Dataset

Click through rate = (#click / #impression ) * 100

PHONEREQUESTS: 
IPID - anonymized identifier of visitor's IP. => is it different from UserID ? 





### Conjecture

Qu'est-ce qui fait qu'on clique sur une ad de context ?  
On clique si l'ad est personnalisée, si elle est adaptée à ce que je recherche ou ce que je suis.  
Ex : pub pour des couches pour une femme enceinte.  
Comment le match entre l'attente de l'utilisateur et le contenu se fait ? L'ad doit être en lien avec les caractéristiques de l'utilsateur. Elle doit probablement être proche de l'intitulé de la recherche ou des filtres mis en place pdt la recherche.  


Un autre facteur dans la décision probable, c'est le profil de l'utilisateur. Certains clique volontier sur des offres de context, d'autres non.  

Le type de la pub joue intrinsèquement un rôle. Ex une pub pour smartphone est + populaire qu'une pub pour un fer à cheval. 

isClick ~ HistCTR + ... + AdsInfo.price + 


### Analyse
=> analyser les recherches 
    quelles recherches sont les + populaires ? 
    ça implique p-e de traduire les queries

=> analyser les ads  
    quelles ads sont les + populaires  
        + d'impression  
        + de clicks
Il doit y avoir une forte correlation entre la top add de la recherche et l'ad context 
    
=> analyser les users  
    quels utilisateurs font le + de recherche ?  
    est-ce qu'on peut dresser des profils d'utilisateurs ? En fonction de leurs envie ?
    est-ce que certaines zones ont des profils d'utilisateurs avec des envies spécifiques ? 
 
    
Traduire : query dans SearchInfos + les categories


In [3]:
! sudo mount -t drvfs f: /mnt/f

[sudo] password for francois: 


In [30]:
ROOT_DIR = '/mnt/f/avito-kaggle/'
SAMPLE_SIZE = 1e4

In [31]:
search_info = pd.read_csv(f"{ROOT_DIR}SearchInfo.tsv", nrows=SAMPLE_SIZE, delimiter="\t", encoding="utf-8-sig"
)

In [5]:
! ls /mnt/f/avito-kaggle

7zfiles       Location.tsv		  UserInfo.tsv
AdsInfo.tsv   PhoneRequestsStream.tsv.7z  VisitsStream.tsv
Category.tsv  SearchInfo.tsv		  avito-context-ad-clicks.zip


In [8]:
location = pd.read_csv(f'{ROOT_DIR}location.tsv', sep='\t')
user_info = pd.read_csv(f'{ROOT_DIR}userInfo.tsv', sep='\t')
category = pd.read_csv(f'{ROOT_DIR}category.tsv', sep='\t')

In [32]:
ads_info = pd.read_csv(f'{ROOT_DIR}AdsInfo.tsv', sep='\t', nrows=SAMPLE_SIZE)

In [33]:
visits_stream = pd.read_csv(f'{ROOT_DIR}VisitsStream.tsv', sep='\t', nrows=SAMPLE_SIZE)

In [23]:
phone_requests_stream = pd.read_csv(f'{ROOT_DIR}PhoneRequestsStream.tsv', sep='\t')

In [34]:
train_search_stream = pd.read_csv(f'{ROOT_DIR}trainSearchStream.tsv', sep='\t', nrows=SAMPLE_SIZE)
test_search_stream = pd.read_csv(f'{ROOT_DIR}testSearchStream.tsv', sep='\t', nrows=SAMPLE_SIZE)

## Data analysis

### Missing values

In [47]:
def show_missing_values(df):
    print(df.name)
    print(df.isna().sum())
    print(f'Total missing values: {df.isna().sum().sum()}\n')

In [44]:
# 8 tables => 9 df (search stream df divided train/test)
search_info.name = "search_info" #échantillonné
location.name = "location"
user_info.name = "user_info"
category.name = "category"
ads_info.name = "ads_info" #échantillonné
visits_stream.name = "visits_stream" #échantillonné
phone_requests_stream.name = "phone_requests_stream" 
train_search_stream.name = "train_search_stream" #échantillonné
test_search_stream.name = "test_search_stream" #échantillonné

dfs = dict(
    search_info = search_info,
    location = location, 
    user_info = user_info, 
    category = category, 
    ads_info = ads_info,
    visits_stream = visits_stream, 
    phone_requests_stream = phone_requests_stream, 
    train_search_stream = train_search_stream, 
    test_search_stream = test_search_stream
)

In [48]:
for df in dfs.values():
    show_missing_values(df)

search_info
SearchID             0
SearchDate           0
IPID                 0
UserID               0
IsUserLoggedOn       0
SearchQuery       8438
LocationID           0
CategoryID           0
SearchParams      5128
dtype: int64
Total missing values: 13566

location
LocationID    0
Level         0
RegionID      1
CityID        1
dtype: int64
Total missing values: 2

user_info
UserID               0
UserAgentID          0
UserAgentOSID        0
UserDeviceID         0
UserAgentFamilyID    0
dtype: int64
Total missing values: 0

category
CategoryID          0
Level               0
ParentCategoryID    0
SubcategoryID       0
dtype: int64
Total missing values: 0

ads_info
AdID            0
LocationID     12
CategoryID      0
Params        261
Price           0
Title           0
IsContext       0
dtype: int64
Total missing values: 273

visits_stream
UserID      0
IPID        0
AdID        0
ViewDate    0
dtype: int64
Total missing values: 0

phone_requests_stream
UserID              0
IPI

SEARCH INFO :  
Pas de surprise pour ce dataframe les variables searchquery et searchparams présentent des valeurs manquantes. Le reste non  
Pour rappel :  
- SearchParams dictionary-like structure with optional filters of the search if they were specified while search. NULL otherwise.
- SearchQuery - raw query text if it was specified while search. NULL otherwise.

LOCATION :
2 NA sur une seule ligne

In [55]:
location[location.isna().any(axis=1)] #bizarrement pour cette ligne on a pas de regionID ni de CityID. Pour l'instant je la laisse

,LocationID,Level,RegionID,CityID
2483,2885,1,NaN,NaN


ADS INFO (échantillonné) :  
LocationID présente des valeurs manquante (peu). En revanche sans surprise Params présente de nombreuses valeurs manquantes.  
Pour rappel : Params parameters for the context ad in dictionary-like format. NULL for some ads (where no parameters are available). Ex : gender, size , ...  
Note that both Params from AdsInfo.tsv and SearchParams from SearchInfo.tsv shares same dictionary (keys and values).
=> pas de modifs

TRAIN & TEST SEARCH :
Only HistCTR qui présente des valeurs manquantes. Parait normal aussi => pas de modifs

### Distribution de la variable cible - isClick

Essayons de voir si on ne peut pas la charger en entier sans faire planter la RAM

In [68]:
y = pd.read_csv(f'{ROOT_DIR}trainSearchStream.tsv', sep='\t', usecols=["IsClick"], squeeze=True)

In [71]:
tmp = y[:100]

In [72]:
y.value_counts()

0.0    189011446
1.0      1146289
Name: IsClick, dtype: int64

In [69]:
def plot_binary_distribution(y):
    # Count the number of occurrences of each class in y
    class_counts = y.value_counts()

    # Get the names of the classes
    class_names = class_counts.index

    # Set the figure size
    plt.figure(figsize=(5,5))

    # Create a pie chart to display the distribution of the classes
    plt.pie(class_counts, labels=class_names, autopct='%1.1f%%', shadow=True, startangle=90)

    # Add a title
    plt.title('Distribution of IsClick Variable')

    # Display the plot
    plt.show()

In [84]:
plot_binary_distribution(y)

KeyboardInterrupt: 